# Random Forest Classification

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## All CpG

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\Male\GeneCpG\UPSIT
df = pd.read_csv('UPSIT3Categ_Methylome_Male.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\NonMotors\Derived250\Male\GeneCpG\UPSIT
(307, 5679)


,Sentrix,PATNO,HYS,Gender,APPRDX,UPSIT_3,UPSIT_Total,cg06067658,cg15442538,cg25928446,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,200991620021_R01C01,3001,2,1,1,1,25,0.088679,0.890148,0.719238,...,0.748763,0.830052,0.861102,0.855659,0.788271,0.805084,0.729165,0.803183,0.925666,0.849608
1,200973410167_R04C01,3007,2,1,1,2,18,0.128668,0.916448,0.726893,...,0.886279,0.898054,0.896800,0.911732,0.863749,0.893595,0.829444,0.866504,0.917233,0.786606
2,200973410167_R05C01,3010,2,1,1,2,9,0.106438,0.922987,0.715904,...,0.847166,0.880457,0.909545,0.902309,0.848555,0.881235,0.840847,0.886343,0.912116,0.786117


In [3]:
df1 = df.drop(['Sentrix','PATNO','HYS','Gender','UPSIT_3','UPSIT_Total'], axis=1)
df1.head(2)

,APPRDX,cg06067658,cg15442538,cg25928446,cg12415575,cg26358225,cg19058526,cg20195763,cg23725394,cg27107076,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,1,0.088679,0.890148,0.719238,0.066678,0.151532,0.133249,0.479724,0.926745,0.538423,...,0.748763,0.830052,0.861102,0.855659,0.788271,0.805084,0.729165,0.803183,0.925666,0.849608
1,1,0.128668,0.916448,0.726893,0.066768,0.198886,0.141136,0.412058,0.876792,0.665097,...,0.886279,0.898054,0.896800,0.911732,0.863749,0.893595,0.829444,0.866504,0.917233,0.786606


In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(50))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.45
2
Mean accuracy: 0.52
3
Mean accuracy: 0.45999999999999996
4
Mean accuracy: 0.48999999999999994
5
Mean accuracy: 0.47000000000000003
6
Mean accuracy: 0.52
7
Mean accuracy: 0.5299999999999999
8
Mean accuracy: 0.5599999999999999
9
Mean accuracy: 0.57
10
Mean accuracy: 0.51
11
Mean accuracy: 0.5599999999999999
12
Mean accuracy: 0.5700000000000001
13
Mean accuracy: 0.47000000000000003
14
Mean accuracy: 0.63
15
Mean accuracy: 0.42000000000000004
16
Mean accuracy: 0.49000000000000005
17
Mean accuracy: 0.5199999999999999
18
Mean accuracy: 0.5
19
Mean accuracy: 0.56
20
Mean accuracy: 0.43000000000000005
21
Mean accuracy: 0.58
22
Mean accuracy: 0.49000000000000005
23
Mean accuracy: 0.56
24
Mean accuracy: 0.6
25
Mean accuracy: 0.51
26
Mean accuracy: 0.5199999999999999
27
Mean accuracy: 0.5200000000000001
28
Mean accuracy: 0.51
29
Mean accuracy: 0.4699999999999999
30
Mean accuracy: 0.4999999999999999
31
Mean accuracy: 0.4
32
Mean accuracy: 0.5200000000000001
33
Mean accuracy

In [5]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("UPSIT_AllCpG_Male_100RF.csv", index = False)

0.5091


## RanFor 1000 times. Samples = 50

In [6]:
df2 = df1.drop (['APPRDX'], axis=1)

In [7]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(50))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.47000000000000003
2
Mean accuracy: 0.5399999999999999
3
Mean accuracy: 0.51
4
Mean accuracy: 0.51
5
Mean accuracy: 0.55
6
Mean accuracy: 0.41999999999999993
7
Mean accuracy: 0.49000000000000005
8
Mean accuracy: 0.45999999999999996
9
Mean accuracy: 0.45
10
Mean accuracy: 0.43
11
Mean accuracy: 0.4800000000000001
12
Mean accuracy: 0.44000000000000006
13
Mean accuracy: 0.4600000000000001
14
Mean accuracy: 0.53
15
Mean accuracy: 0.52
16
Mean accuracy: 0.45999999999999996
17
Mean accuracy: 0.44000000000000006
18
Mean accuracy: 0.63
19
Mean accuracy: 0.51
20
Mean accuracy: 0.49000000000000005
21
Mean accuracy: 0.53
22
Mean accuracy: 0.54
23
Mean accuracy: 0.48999999999999994
24
Mean accuracy: 0.52
25
Mean accuracy: 0.51
26
Mean accuracy: 0.58
27
Mean accuracy: 0.6
28
Mean accuracy: 0.6
29
Mean accuracy: 0.54
30
Mean accuracy: 0.43
31
Mean accuracy: 0.49000000000000005
32
Mean accuracy: 0.65
33
Mean accuracy: 0.53
34
Mean accuracy: 0.48999999999999994
35
Mean accuracy: 0.46

In [8]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("UPSIT_Rand50CpG_Male_100RF.csv", index = False)

0.4999000000000002


## RanFor 100 times. Samples = 96

In [9]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(96, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(50))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.41000000000000003
2
Mean accuracy: 0.53
3
Mean accuracy: 0.5199999999999999
4
Mean accuracy: 0.47000000000000003
5
Mean accuracy: 0.5599999999999999
6
Mean accuracy: 0.5
7
Mean accuracy: 0.43999999999999995
8
Mean accuracy: 0.45
9
Mean accuracy: 0.44000000000000006
10
Mean accuracy: 0.36
11
Mean accuracy: 0.42000000000000004
12
Mean accuracy: 0.43999999999999995
13
Mean accuracy: 0.4
14
Mean accuracy: 0.47000000000000003
15
Mean accuracy: 0.43
16
Mean accuracy: 0.48
17
Mean accuracy: 0.44000000000000006
18
Mean accuracy: 0.6399999999999999
19
Mean accuracy: 0.5900000000000001
20
Mean accuracy: 0.4699999999999999
21
Mean accuracy: 0.5
22
Mean accuracy: 0.51
23
Mean accuracy: 0.54
24
Mean accuracy: 0.48
25
Mean accuracy: 0.4600000000000001
26
Mean accuracy: 0.53
27
Mean accuracy: 0.51
28
Mean accuracy: 0.5799999999999998
29
Mean accuracy: 0.5399999999999999
30
Mean accuracy: 0.51
31
Mean accuracy: 0.61
32
Mean accuracy: 0.53
33
Mean accuracy: 0.45999999999999996
34
Mea

In [10]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("UPSIT_Rand96CpG_Male_100RF.csv", index = False)

0.49770000000000003


## RanFor Top 50 Boruta 100 times. Samples = 50

In [11]:
df = pd.read_csv('UPSIT_Boruta_Top50_Data_Male.csv')
print(df.shape)
df.head(2)

(307, 52)


,PATNO,APPRDX,cg00260798,cg11023668,cg00877792,cg27498387,cg01458259,cg05294431,cg19997384,cg14927724,...,cg24628866,cg15323638,cg11559967,cg24718866,cg07806248,cg10210919,cg15028128,cg07897734,cg19731340,cg24272338
0,3001,1,0.885679,0.497315,0.497734,0.316042,0.820166,0.726805,0.284672,0.284245,...,0.918421,0.174982,0.896292,0.013781,0.934287,0.026093,0.921116,0.842938,0.018411,0.912405
1,3007,1,0.916535,0.653975,0.453499,0.250261,0.857082,0.824456,0.329010,0.258556,...,0.915580,0.172641,0.918307,0.012835,0.905446,0.023069,0.919847,0.923446,0.017375,0.891393


In [12]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg00260798,cg11023668,cg00877792,cg27498387,cg01458259,cg05294431,cg19997384,cg14927724,cg22219083,...,cg24628866,cg15323638,cg11559967,cg24718866,cg07806248,cg10210919,cg15028128,cg07897734,cg19731340,cg24272338
0,1,0.885679,0.497315,0.497734,0.316042,0.820166,0.726805,0.284672,0.284245,0.018263,...,0.918421,0.174982,0.896292,0.013781,0.934287,0.026093,0.921116,0.842938,0.018411,0.912405
1,1,0.916535,0.653975,0.453499,0.250261,0.857082,0.824456,0.329010,0.258556,0.021999,...,0.915580,0.172641,0.918307,0.012835,0.905446,0.023069,0.919847,0.923446,0.017375,0.891393


In [13]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(50))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.6199999999999999
2
Mean accuracy: 0.6799999999999999
3
Mean accuracy: 0.6599999999999999
4
Mean accuracy: 0.6599999999999999
5
Mean accuracy: 0.58
6
Mean accuracy: 0.6100000000000001
7
Mean accuracy: 0.7
8
Mean accuracy: 0.73
9
Mean accuracy: 0.7700000000000001
10
Mean accuracy: 0.6399999999999999
11
Mean accuracy: 0.6299999999999999
12
Mean accuracy: 0.6699999999999999
13
Mean accuracy: 0.69
14
Mean accuracy: 0.7
15
Mean accuracy: 0.64
16
Mean accuracy: 0.59
17
Mean accuracy: 0.6
18
Mean accuracy: 0.6399999999999999
19
Mean accuracy: 0.6299999999999999
20
Mean accuracy: 0.64
21
Mean accuracy: 0.7100000000000001
22
Mean accuracy: 0.6900000000000001
23
Mean accuracy: 0.74
24
Mean accuracy: 0.7
25
Mean accuracy: 0.72
26
Mean accuracy: 0.71
27
Mean accuracy: 0.72
28
Mean accuracy: 0.68
29
Mean accuracy: 0.68
30
Mean accuracy: 0.69
31
Mean accuracy: 0.6799999999999999
32
Mean accuracy: 0.75
33
Mean accuracy: 0.6900000000000001
34
Mean accuracy: 0.6799999999999999
35
Mean

In [14]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("UPSIT_Boruta50CpG_Male_100RF.csv", index = False)

0.6710999999999998


## RanFor Top 50 NMI 100 times. Samples = 50

In [15]:
df = pd.read_csv('UPSIT_NMI_Top50_Male_Data.csv')
print(df.shape)
df.head(2)

(307, 51)


,APPRDX,cg26358225,cg01605403,cg22295787,cg02295000,cg00360077,cg02176314,cg14746118,cg23074747,cg12501949,...,cg24260971,cg27315341,cg06263834,cg02944903,cg16628215,cg20003494,cg05142211,cg19646414,cg07466585,cg23127998
0,1,0.151532,0.879499,0.016057,0.826318,0.912095,0.916863,0.778754,0.011630,0.747517,...,0.101776,0.855311,0.049564,0.779978,0.053833,0.025708,0.035018,0.888881,0.082290,0.216844
1,1,0.198886,0.935782,0.021968,0.805755,0.916703,0.920714,0.798111,0.011863,0.757032,...,0.060896,0.769311,0.030237,0.653859,0.022553,0.045513,0.041906,0.885429,0.025043,0.166560


In [16]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df.groupby('APPRDX').apply(lambda x: x.sample(50))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.58
2
Mean accuracy: 0.67
3
Mean accuracy: 0.6499999999999999
4
Mean accuracy: 0.68
5
Mean accuracy: 0.5799999999999998
6
Mean accuracy: 0.6799999999999999
7
Mean accuracy: 0.62
8
Mean accuracy: 0.7
9
Mean accuracy: 0.6
10
Mean accuracy: 0.68
11
Mean accuracy: 0.7
12
Mean accuracy: 0.6299999999999999
13
Mean accuracy: 0.6300000000000001
14
Mean accuracy: 0.6299999999999999
15
Mean accuracy: 0.6399999999999999
16
Mean accuracy: 0.51
17
Mean accuracy: 0.62
18
Mean accuracy: 0.63
19
Mean accuracy: 0.6699999999999999
20
Mean accuracy: 0.6799999999999999
21
Mean accuracy: 0.6799999999999999
22
Mean accuracy: 0.72
23
Mean accuracy: 0.64
24
Mean accuracy: 0.69
25
Mean accuracy: 0.7
26
Mean accuracy: 0.6199999999999999
27
Mean accuracy: 0.67
28
Mean accuracy: 0.72
29
Mean accuracy: 0.6100000000000001
30
Mean accuracy: 0.6599999999999999
31
Mean accuracy: 0.59
32
Mean accuracy: 0.63
33
Mean accuracy: 0.7100000000000001
34
Mean accuracy: 0.6199999999999999
35
Mean accuracy: 0.6

In [17]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("UPSIT_NMI50CpG_Male_100RF.csv", index = False)

0.6561


## RanFor Top 50 Boruta + NMI 100 times. Samples = 96

In [18]:
df = pd.read_csv('UPSIT_Top50sMerge_Data_Male.csv')
print(df.shape)
df.head(2)

(307, 98)


,PATNO,APPRDX,cg00260798,cg11023668,cg00877792,cg27498387,cg01458259,cg05294431,cg19997384,cg14927724,...,cg24260971,cg27315341,cg06263834,cg02944903,cg16628215,cg20003494,cg05142211,cg19646414,cg07466585,cg23127998
0,3001,1,0.885679,0.497315,0.497734,0.316042,0.820166,0.726805,0.284672,0.284245,...,0.101776,0.855311,0.049564,0.779978,0.053833,0.025708,0.035018,0.888881,0.082290,0.216844
1,3007,1,0.916535,0.653975,0.453499,0.250261,0.857082,0.824456,0.329010,0.258556,...,0.060896,0.769311,0.030237,0.653859,0.022553,0.045513,0.041906,0.885429,0.025043,0.166560


In [19]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg00260798,cg11023668,cg00877792,cg27498387,cg01458259,cg05294431,cg19997384,cg14927724,cg22219083,...,cg24260971,cg27315341,cg06263834,cg02944903,cg16628215,cg20003494,cg05142211,cg19646414,cg07466585,cg23127998
0,1,0.885679,0.497315,0.497734,0.316042,0.820166,0.726805,0.284672,0.284245,0.018263,...,0.101776,0.855311,0.049564,0.779978,0.053833,0.025708,0.035018,0.888881,0.082290,0.216844
1,1,0.916535,0.653975,0.453499,0.250261,0.857082,0.824456,0.329010,0.258556,0.021999,...,0.060896,0.769311,0.030237,0.653859,0.022553,0.045513,0.041906,0.885429,0.025043,0.166560


In [ ]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(50))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.57
2
Mean accuracy: 0.73
3
Mean accuracy: 0.76
4
Mean accuracy: 0.6
5
Mean accuracy: 0.5899999999999999
6
Mean accuracy: 0.72
7
Mean accuracy: 0.7
8
Mean accuracy: 0.7
9
Mean accuracy: 0.7100000000000001
10
Mean accuracy: 0.6699999999999999
11
Mean accuracy: 0.65
12
Mean accuracy: 0.6499999999999999
13
Mean accuracy: 0.5900000000000001
14
Mean accuracy: 0.7
15
Mean accuracy: 0.6399999999999999
16
Mean accuracy: 0.6799999999999999
17
Mean accuracy: 0.55
18
Mean accuracy: 0.69
19
Mean accuracy: 0.68
20
Mean accuracy: 0.6799999999999999
21
Mean accuracy: 0.6399999999999999
22
Mean accuracy: 0.72
23
Mean accuracy: 0.72
24
Mean accuracy: 0.7
25
Mean accuracy: 0.75
26
Mean accuracy: 0.67
27
Mean accuracy: 0.6599999999999999
28
Mean accuracy: 0.7300000000000001
29
Mean accuracy: 0.63
30
Mean accuracy: 0.66
31
Mean accuracy: 0.6399999999999999
32
Mean accuracy: 0.7100000000000001
33
Mean accuracy: 0.7000000000000001
34
Mean accuracy: 0.6699999999999999
35
Mean accuracy: 0.69

In [ ]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("UPSIT_Top50sCpG96_Male_100RF.csv", index = False)